In [1]:
from Preprocessing.data_clean import remove_221_rows, dropping_no_betting_data, josh_features, fill_f_pm_01m, class_or_rating_average, oli_features
from Preprocessing.v2_preprocessor_js import preprocess_features_v2
import pandas as pd

data  = pd.read_csv("../raw_data/raw_data_v2.2.csv")

data_cleaned = remove_221_rows(data)
data_cleaned = dropping_no_betting_data(data_cleaned)
data_cleaned = josh_features(data_cleaned)
data_cleaned = class_or_rating_average(data_cleaned)
data_cleaned = oli_features(data_cleaned)
data_cleaned = fill_f_pm_01m(data_cleaned)



preprocessed_data = preprocess_features_v2(data_cleaned)

preprocessed_data.to_csv("../raw_data/data_cleaned_and_preprocessed.csv", index=False)

data = pd.read_csv("../raw_data/data_cleaned_and_preprocessed.csv")
data.sort_values(by='f_ko')
data = data.reset_index(drop=True)

/var/folders/w6/wdkqbmf17b90n_r35p0cb7x00000gn/T/ipykernel_32677/199618635.py:5: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data  = pd.read_csv("../raw_data/raw_data_v2.2.csv")


first 221 rows removed. New shape = (118354, 116)


/Users/joshstone/code/lucasglanville/and_theyre_off_backend/Preprocessing/data_clean.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_impute] = data[columns_to_impute].apply(impute_row, axis=1)


Cleaned up missing odds. New shape = (118093, 116)
Added Josh features. New shape = (118093, 123)
Added Oli features 2/4. New shape = (118093, 125)
Added Oli features 4/4. New shape = (118093, 127)
number of columns:  128
✅ DROPPED IRRELEVANT COLUMNS
✅ DROPPED ROWS WITH NULL VALUES
✅ WHITESPACE STRIPPED FROM 'f_track'
✅ WINNERS CODED AS '1', REST '0'
✅ 'f_ko' CONVERTED TO DATETIME
✅ TRACK CONDITIONS ORDINALLY ENCODED
✅ NUMERIC FEATURES MINMAX-SCALED
✅ IMPUTED 'no_headgear' for NULLS IN 'f_headgear'
✅ IMPUTED MEAN FOR NULLS IN 'f_dob' & 'f_prb_avg'
✅ IMPUTED '0' FOR NULLS IN 68 x FEATURES
✅ CAT. FEATURES OH-ENCODED (Track, Headgear, Country)
number of columns:  88
✅ COLUMN TRANSFORMER ASSEMBLED
⏳ FIT_TRANSFORMING THE PREPROCESSING PIPE...
number of columns:  164
✅ DATA PROCESSED WITH SHAPE: (117468, 164)


In [2]:
data = data[(data['f_pm_01m'] <= 50)]

In [3]:
data = data.reset_index(drop=True)

In [4]:
data.head(2)

,f_going,average_or_rating_class,above_below_official_rating_class,PreviousPosition,PredictedRank,f_distance,f_class,f_age,f_pace,f_weight,...,f_prb_avg,id,f_id,f_ko,f_horse,pred_isp,f_place,f_pm_01m,f_pm_01m_p_back,linear_target
0,0.4,0.836105,0.643877,0.0,0.000000,0.0,0.166667,0.25,0.7,0.21875,...,0.434968,16149939000396,16149939000030,2021-03-06 01:25:00,Tone The Barone,2.5,0,2.28,-1.0,-1.0
1,0.4,0.836105,0.691321,0.0,0.032258,0.0,0.166667,0.50,0.5,0.34375,...,0.434968,16149939000058,16149939000030,2021-03-06 01:25:00,Blue De Vega,4.5,0,10.00,-1.0,-1.0


In [5]:
backtest = data[['f_ko','f_id', 'id','f_horse','f_pm_01m', 'linear_target', 'f_place']]




data['pred_isp_prob'] = 1 / data['pred_isp']
data['linear_target_prob'] = 1 / data['linear_target']

In [6]:
data.head(2)

,f_going,average_or_rating_class,above_below_official_rating_class,PreviousPosition,PredictedRank,f_distance,f_class,f_age,f_pace,f_weight,...,f_id,f_ko,f_horse,pred_isp,f_place,f_pm_01m,f_pm_01m_p_back,linear_target,pred_isp_prob,linear_target_prob
0,0.4,0.836105,0.643877,0.0,0.000000,0.0,0.166667,0.25,0.7,0.21875,...,16149939000030,2021-03-06 01:25:00,Tone The Barone,2.5,0,2.28,-1.0,-1.0,0.400000,-1.0
1,0.4,0.836105,0.691321,0.0,0.032258,0.0,0.166667,0.50,0.5,0.34375,...,16149939000030,2021-03-06 01:25:00,Blue De Vega,4.5,0,10.00,-1.0,-1.0,0.222222,-1.0


In [7]:

X = data[['stall_position', 
          'iv_trainer_l16r', 'iv_jockey_l16r',
          'ae_trainer_l16r' ,'ae_jockey_l16r', 'pred_isp_prob', 'f_runners']]

# X = data[['f_runners','or_rating_vs_avg_race', 'stall_position', 
#           'iv_horse_at', 'iv_trainer_l200r', 'iv_trainer_l50r', 'iv_trainer_l16r', 'iv_trainer_at',
#           'iv_jockey_l200r', 'iv_jockey_l50r', 'iv_jockey_l16r', 'iv_jockey_at',
#           'ae_horse_l10r', 'ae_horse_l5r' , 'ae_horse_l2r' ,'ae_horse_at',
#           'ae_trainer_l200r' , 'ae_trainer_l50r' , 'ae_trainer_l16r' , 'ae_trainer_at',
#           'ae_jockey_l200r' , 'ae_jockey_l50r', 'ae_jockey_l16r' , 'ae_jockey_at' , 'pred_isp']]

#'f_dob', 'f_prb_avg',





y = data[["f_place","pred_isp"]] #OR 'linear_target'

print(data.shape)
print(X.shape)
print(y.shape)
print(backtest.shape)

#Train = Year 1
#Val = Year 2
#Test = Year 3 (6 months)

print(X.head(2))
print(y.head(2))

(102875, 166)
(102875, 7)
(102875, 2)
(102875, 7)
   stall_position  iv_trainer_l16r  iv_jockey_l16r  ae_trainer_l16r  \
0        0.148019              0.0             0.5              0.0   
1        0.480479              0.0             0.0              0.0   

   ae_jockey_l16r  pred_isp_prob  f_runners  
0        0.014276       0.400000    0.15625  
1        0.000000       0.222222    0.15625  
   f_place  pred_isp
0        0       2.5
1        0       4.5


In [8]:
X = X.drop(columns = 'pred_isp_prob')

In [9]:
X

,stall_position,iv_trainer_l16r,iv_jockey_l16r,ae_trainer_l16r,ae_jockey_l16r,f_runners
0,0.148019,0.000000,0.500000,0.000000,0.014276,0.15625
1,0.480479,0.000000,0.000000,0.000000,0.000000,0.15625
2,0.231163,1.000000,1.000000,0.094158,0.128913,0.15625
3,0.314249,0.000000,0.000000,0.000000,0.000000,0.15625
4,0.563624,0.000000,0.000000,0.000000,0.000000,0.15625
...,...,...,...,...,...,...
102870,0.116076,0.187629,0.062887,0.015622,0.005418,0.34375
102871,0.339850,0.062887,0.062887,0.004601,0.006536,0.34375
102872,0.295107,0.000000,0.124742,0.000000,0.006708,0.34375
102873,0.160819,0.062887,0.124742,0.005778,0.008944,0.34375


In [10]:
X_train = X.iloc[:40000]
X_val = X.iloc[40000:70000]
X_test = X.iloc[70000:]
y_train = y.iloc[:40000]
y_val = y.iloc[40000:70000]
y_test = y.iloc[70000:]
backtest_train = backtest.iloc[:40000]
backtest_val = backtest.iloc[40000:70000]
backtest_test = backtest.iloc[70000:]

In [15]:
X_val.shape

(30000, 6)

In [20]:
X_test.to_csv("NN1_X_test.csv", index = False)

In [21]:
y_test.to_csv("NN1_y_test.csv", index = False)

In [22]:
backtest_test.to_csv("NN1_backtest_test.csv", index = False)

In [18]:
backtest_test.to_csv("NN1_backtest_test.csv")

In [ ]:
#buggy data = 2532754    2021-06-20 03:55:00
                #29406    20-06-14 18:45:00

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_test.head()

In [ ]:
y_test.shape

In [ ]:
X_test.to_csv("NN2_X_test_7.csv", index = False)

In [ ]:
y_test.to_csv("NN2_y_test_7.csv", index = False)

In [ ]:
backtest_test.to_csv("NN2_backtest_test_7.csv", index = False)

In [ ]:
# def odds_loss(y_true, y_pred):
#     """
#     The function implements the custom loss function
    
#     Inputs
#     true : a vector of dimension batch_size, 7. A label encoded version of the output and the backp1_a and backp1_b
#     pred : a vector of probabilities of dimension batch_size , 5.
    
#     Returns 
#     the loss value
#     """
#     win_home_team = y_true[:, 0:1]
#     win_home_or_draw = y_true[:, 1:2]
#     win_away = y_true[:, 2:3]
#     win_away_or_draw = y_true[:, 3:4]
#     draw = y_true[:, 4:5]
#     no_bet = y_true[:, 5:6]
#     odds_a = y_true[:, 6:7]
#     odds_b = y_true[:, 7:8]
#     gain_loss_vector = K.concatenate([win_home_team * (odds_a - 1) + (1 - win_home_team) * -1,
#       win_home_or_draw * (1/(1 -1/odds_b) - 1) + (1 - win_home_or_draw) * -1,
#       win_away * (odds_b - 1) + (1 - win_away) * -1,
#       win_away_or_draw * (1/(1 -1/odds_a) - 1) + (1 - win_away_or_draw) * -1,
#       draw * (1/(1 - 1/odds_a - 1/odds_b) - 1) + (1 - draw) * -1,
#       K.zeros_like(odds_a)], axis=1)
#     return -1 * K.mean(K.sum(gain_loss_vector * y_pred, axis=1))

In [ ]:
# def get_data():
#     data = pd.read_csv('extract-betsentiment-com.csv')
#     X = data.values[:, 5:-5]
#     y = data.values[:, -1]
#     y_full = np.zeros((X.shape[0], 8))
#     for i, y_i in enumerate(y):
#         if y_i == 1:
#             y_full[i, 0] = 1.0
#             y_full[i, 1] = 1.0
#         if y_i == 2:
#             y_full[i, 2] = 1.0
#             y_full[i, 3] = 1.0
#         if y_i == 3:
#             y_full[i, 1] = 1.0
#             y_full[i, 3] = 1.0
#             y_full[i, 4] = 1.0
#         y_full[i, 6] = X[i, 1] # ADD ODDS OF HOME TEAM
#         y_full[i, 7] = X[i, 2] # ADD ODDS OF AWAY TEAM
#     return X, y_full, y
# X, y, outcome = get_data()
# # SPLIT THE DATA IN TRAIN AND TEST DATASET.
# train_x, test_x, train_y, test_y, = train_test_split(X,  y)

In [ ]:
from keras.utils import get_custom_objects
import tensorflow as tf
from keras import backend as K


def custom_loss_function(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)  # Convert y_true to float32
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    
    win_horse = y_true[:, 0:1]
    win_odds = y_true[:, 1:2]
    gain_loss_vector = K.concatenate([win_horse * (win_odds - 1) + (1 - win_horse) * -1,
    K.zeros_like(win_odds)], axis=1)
    return -0.1 * K.mean(K.sum(gain_loss_vector * y_pred, axis=1))
    #return -1 * tf.reduce_mean(tf.reduce_sum(gain_loss_vector * y_pred, axis=1))

get_custom_objects().update({"custom_loss_function": custom_loss_function})

In [ ]:
# convert labels to a one-hot vector
# from tensorflow.keras.utils import to_categorical

# train_labels = to_categorical(train_labels)
# test_labels = to_categorical(test_labels)

# define network architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Precision, Accuracy, Recall, F1Score
from tensorflow.keras.losses import Poisson
from tensorflow.keras.optimizers import Adam

MLP2 = Sequential()
MLP2.add(InputLayer(input_shape=(7, ))) # input layer
MLP2.add(Dense(32, activation='relu')) # hidden layer 1
MLP2.add(Dense(2, activation='softmax')) # output layer


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import InputLayer
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.metrics import Precision, Accuracy, Recall, F1Score
# from tensorflow.keras.losses import Poisson
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras import Model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# def get_model(input_dim, output_dim, base=1000, multiplier=0.25, p=0.2):
#     inputs = Input(shape=(input_dim,))
#     l = BatchNormalization()(inputs)
#     l = Dropout(p)(l)
#     n = base
#     l = Dense(n, activation='relu')(l)
#     l = BatchNormalization()(l)
#     l = Dropout(p)(l)
#     n = int(n * multiplier)
#     l = Dense(n, activation='relu')(l)
#     l = BatchNormalization()(l)
#     l = Dropout(p)(l)
#     n = int(n * multiplier)
#     l = Dense(n, activation='relu')(l)
#     outputs = Dense(output_dim, activation='softmax')(l)
#     model = Model(inputs=inputs, outputs=outputs)
#     model.compile(optimizer='Nadam', loss=custom_loss_function)
#     return model
# model = get_model(7, 2, 1000, 0.5, 0.2)
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
#           epochs=200, batch_size=5, callbacks=[EarlyStopping(patience=2),ModelCheckpoint('odds_loss.hdf5',save_best_only=True)])

In [ ]:
# optimization
MLP2.compile(loss='custom_loss_function',
            optimizer=Adam(learning_rate = 0.0005))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# train (fit)

es = EarlyStopping(patience=5, restore_best_weights=True)

history = MLP2.fit(X_train, y_train, 
                  epochs=200, batch_size=32, verbose=1,
                  validation_data=(X_val, y_val), callbacks = [es], shuffle = True)

In [ ]:
# evaluate performance on test data
a = MLP2.evaluate(X_test, y_test,
                                         verbose=1)
a

In [ ]:
y_pred = MLP2.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred[:, 0]

In [ ]:
import seaborn as sns
sns.histplot(y_pred[:, 0:1], bins = 50, kde=True)

In [ ]:
backtest_test['model_preds'] = y_pred[:, 0:1]
backtest_test['odds_preds'] = 1/ backtest_test['f_pm_01m']
backtest_test['model_edge_basic'] = backtest_test['model_preds'] - backtest_test['odds_preds']
backtest_test['model_edge_proportionate'] = backtest_test['model_preds'] / backtest_test['odds_preds']

In [ ]:
#backtest_test = backtest_test.drop(columns=['f_ko'])
model_preds_race = backtest_test.groupby('f_id').sum().reset_index()
model_preds_race2 = model_preds_race[['f_id', 'model_preds', 'odds_preds']]
dict_odds_preds = model_preds_race2.set_index('f_id')['odds_preds'].to_dict()
dict_race_preds = model_preds_race2.set_index('f_id')['model_preds'].to_dict()
backtest_test['race_model_preds'] = backtest_test['f_id'].map(dict_race_preds)
backtest_test['race_odds_preds'] = backtest_test['f_id'].map(dict_odds_preds)
backtest_test['model_preds_scaled'] = (backtest_test['model_preds'] / backtest_test['race_model_preds']) * backtest_test['race_odds_preds']
backtest_test['model_edge_basic_scaled'] = backtest_test['model_preds_scaled'] - backtest_test['odds_preds']
backtest_test['model_edge_prop_scaled'] = backtest_test['model_preds_scaled'] / backtest_test['odds_preds']

In [ ]:
backtest_test['model_preds'] = round(backtest_test['model_preds'],2)

In [ ]:
backtest_test.head(2)

In [ ]:
# Create dataframes of horses above specific proba_difference thresholds

all_bets = backtest_test
above0 = backtest_test[(backtest_test['model_preds'] >= 0.5) & (backtest_test['f_pm_01m'] < 2)]
above5 = backtest_test[(backtest_test['model_preds'] >= 0.5) & (backtest_test['f_pm_01m'] >= 2) & (backtest_test['f_pm_01m'] < 5)]
above10 = backtest_test[(backtest_test['model_preds'] >= 0.5) & (backtest_test['f_pm_01m'] >= 5) & (backtest_test['f_pm_01m'] < 10)]
above15 = backtest_test[(backtest_test['model_preds'] >= 0.5) & (backtest_test['f_pm_01m'] >= 10) & (backtest_test['f_pm_01m'] < 25)]
above20 = backtest_test[(backtest_test['model_preds'] >= 0.5) & (backtest_test['f_pm_01m'] >= 25) & (backtest_test['f_pm_01m'] < 51)]
above30 = backtest_test[backtest_test['model_preds'] >= 0.5]

# Calculate evolutions of profits for each threshold

dfall = all_bets
dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

df0 = above0
df0['cumulative_profit'] = df0['linear_target'].cumsum()

df5 = above5
df5['cumulative_profit'] = df5['linear_target'].cumsum()

df10 = above10
df10['cumulative_profit'] = df10['linear_target'].cumsum()

df15 = above15
df15['cumulative_profit'] = df15['linear_target'].cumsum()

df20 = above20
df20['cumulative_profit'] = df20['linear_target'].cumsum()

df30 = above30
df30['cumulative_profit'] = df30['linear_target'].cumsum()

import matplotlib.pyplot as plt
# Plot
plt.figure(figsize=(10,6))
#plt.plot(range(1, len(dfall) + 1), dfall['cumulative_profit'], label='all bets')
plt.plot(range(1, len(df0) + 1), df0['cumulative_profit'], label='<2 odds')
plt.plot(range(1, len(df5) + 1), df5['cumulative_profit'], label='2-5 odds')
plt.plot(range(1, len(df10) + 1), df10['cumulative_profit'], label='5-10 odds')
plt.plot(range(1, len(df15) + 1), df15['cumulative_profit'], label='10-25 odds')
plt.plot(range(1, len(df20) + 1), df20['cumulative_profit'], label='25-50 odds')
#plt.plot(range(1, len(df30) + 1), df30['cumulative_profit'], label='0.875 threshold')

plt.title("xxx")
plt.xlabel("number of bets")
plt.ylabel("profit (£)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history["val_loss"])
plt.show()

In [ ]:
#MLP2.save_weights("custom_scorer0.05_7input_l16")

In [ ]:
# Create dataframes of horses above specific proba_difference thresholds

all_bets = backtest_test
above0 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0]
above5 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.025]
above10 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.05]
above15 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.075]
above20 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.10]
above30 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.20]

# Calculate evolutions of profits for each threshold

dfall = all_bets
dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

df0 = above0
df0['cumulative_profit'] = df0['linear_target'].cumsum()

df5 = above5
df5['cumulative_profit'] = df5['linear_target'].cumsum()

df10 = above10
df10['cumulative_profit'] = df10['linear_target'].cumsum()

df15 = above15
df15['cumulative_profit'] = df15['linear_target'].cumsum()

df20 = above20
df20['cumulative_profit'] = df20['linear_target'].cumsum()

df30 = above30
df30['cumulative_profit'] = df30['linear_target'].cumsum()

In [ ]:
# Create dataframes of horses above specific proba_difference thresholds

all_bets = backtest_test
above0b = backtest_test[backtest_test['model_edge_scaled'] >= 0]
above5b = backtest_test[backtest_test['model_edge_scaled'] >= 0.05]
above10b = backtest_test[backtest_test['model_edge_scaled'] >= 0.1]
above15b = backtest_test[backtest_test['model_edge_scaled'] >= 0.15]
above20b = backtest_test[backtest_test['model_edge_scaled'] >= 0.20]
above30b = backtest_test[backtest_test['model_edge_scaled'] >= 0.30]

# Calculate evolutions of profits for each threshold

dfall = all_bets
dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

df0b = above0b
df0b['cumulative_profit'] = df0b['linear_target'].cumsum()

df5b = above5b
df5b['cumulative_profit'] = df5b['linear_target'].cumsum()

df10b = above10b
df10b['cumulative_profit'] = df10b['linear_target'].cumsum()

df15b = above15b
df15b['cumulative_profit'] = df15b['linear_target'].cumsum()

df20b = above20b
df20b['cumulative_profit'] = df20b['linear_target'].cumsum()

df30b = above30b
df30b['cumulative_profit'] = df30b['linear_target'].cumsum()

In [ ]:
import matplotlib.pyplot as plt
# Plot
plt.figure(figsize=(10,6))
#plt.plot(range(1, len(dfall) + 1), dfall['cumulative_profit'], label='all bets')
#plt.plot(range(1, len(df0) + 1), df0['cumulative_profit'], label='0 threshold')
plt.plot(range(1, len(df5) + 1), df5['cumulative_profit'], label='2.5 threshold')
plt.plot(range(1, len(df10) + 1), df10['cumulative_profit'], label='5 threshold')
plt.plot(range(1, len(df15) + 1), df15['cumulative_profit'], label='7.5 threshold')
plt.plot(range(1, len(df20) + 1), df20['cumulative_profit'], label='10 threshold')
plt.plot(range(1, len(df30) + 1), df30['cumulative_profit'], label='20 threshold')
#plt.plot(range(1, len(df5b) + 1), df5b['cumulative_profit'], label='5 threshold unscaled')
#plt.plot(range(1, len(df10b) + 1), df10b['cumulative_profit'], label='10 threshold unscaled')
#plt.plot(range(1, len(df15b) + 1), df15b['cumulative_profit'], label='15 threshold unscaled')
#plt.plot(range(1, len(df20b) + 1), df20b['cumulative_profit'], label='20 threshold unscaled')
#plt.plot(range(1, len(df30b) + 1), df30b['cumulative_profit'], label='30 threshold unscaled')

plt.title("xxx")
plt.xlabel("number of bets")
plt.ylabel("profit (£)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.hist(backtest_test['model_edge_proportionate'])

In [ ]:
# Create dataframes of horses above specific proba_difference thresholds

all_bets = backtest_test
above0c = backtest_test[backtest_test['model_edge_proportionate'] >= 1]
above5c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.125]
above10c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.25]
above15c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.375]
above20c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.5]
above30c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.75]

# Calculate evolutions of profits for each threshold

dfall = all_bets
dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

df0c = above0c
df0c['cumulative_profit'] = df0c['linear_target'].cumsum()

df5c = above5c
df5c['cumulative_profit'] = df5c['linear_target'].cumsum()

df10c = above10c
df10c['cumulative_profit'] = df10c['linear_target'].cumsum()

df15c = above15c
df15c['cumulative_profit'] = df15c['linear_target'].cumsum()

df20c = above20c
df20c['cumulative_profit'] = df20c['linear_target'].cumsum()

df30c = above30c
df30c['cumulative_profit'] = df30c['linear_target'].cumsum()

In [ ]:
# Create dataframes of horses above specific proba_difference thresholds

all_bets = backtest_test
above0d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1]
above5d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.05]
above10d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.1]
above15d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.15]
above20d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.2]
above30d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.25]

# Calculate evolutions of profits for each threshold

dfall = all_bets
dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

df0d = above0d
df0d['cumulative_profit'] = df0c['linear_target'].cumsum()

df5d = above5d
df5d['cumulative_profit'] = df5c['linear_target'].cumsum()

df10d = above10d
df10d['cumulative_profit'] = df10c['linear_target'].cumsum()

df15d = above15d
df15d['cumulative_profit'] = df15c['linear_target'].cumsum()

df20d = above20d
df20d['cumulative_profit'] = df20c['linear_target'].cumsum()

df30d = above30d
df30d['cumulative_profit'] = df30c['linear_target'].cumsum()

In [ ]:
import matplotlib.pyplot as plt
# Plot
plt.figure(figsize=(10,6))
#plt.plot(range(1, len(dfall) + 1), dfall['cumulative_profit'], label='all bets')
#plt.plot(range(1, len(df0d) + 1), df0d['cumulative_profit'], label='1 threshold')
#plt.plot(range(1, len(df5d) + 1), df5d['cumulative_profit'], label='1.125 threshold')
plt.plot(range(1, len(df10d) + 1), df10d['cumulative_profit'], label='1.25 threshold')
plt.plot(range(1, len(df15d) + 1), df15d['cumulative_profit'], label='1.375 threshold')
plt.plot(range(1, len(df20d) + 1), df20d['cumulative_profit'], label='1.5 threshold')
plt.plot(range(1, len(df30d) + 1), df30d['cumulative_profit'], label='1.75 threshold')
#plt.plot(range(1, len(df5c) + 1), df5c['cumulative_profit'], label='1.05 threshold unscaled')
#plt.plot(range(1, len(df10c) + 1), df10c['cumulative_profit'], label='1.1 threshold unscaled')
#plt.plot(range(1, len(df15c) + 1), df15c['cumulative_profit'], label='1.15 threshold unscaled')
#plt.plot(range(1, len(df20c) + 1), df20c['cumulative_profit'], label='1.2 threshold unscaled')
#plt.plot(range(1, len(df30c) + 1), df30c['cumulative_profit'], label='1.25 threshold unscaled')

plt.title("xxx")
plt.xlabel("number of bets")
plt.ylabel("profit (£)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
backtest_test